In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

from sinusoidal import SinusoidSignal

plt.style.use("bmh")

# Phasors Visualized

The goal of this notebook is to visually represent phasors so that they are intuitive to understand. Hopefully they seem like an obviously good choice for sinusoidal signals. 

The notebook is structured as follows:
1. A Derivation of Euler's Equation
2. Modeling a single sinusoid
3. Modeling multiple sinusoids
4. Modeling a sinusoidal wave interfering with its reflection

## 1) A Derivation of Euler's Equation
Before talking about sinusoids or time domain signals, let's start with coming to grips with Euler's equation. Let's start by asking the question, what does it mean to raise a number to the power of an imaginary number?

$$e^i = ???$$

We will introduce two methods of deriving euler's equations. The first is maybe a bit less intuitive but is more rigorous. 

### I) Euler's Equation from Taylor Series
We start by defining a function $f(x)$ which is equal to its derivative:

$$f(x) = f'(x)$$

You probably already remember what the answer is. But let's try to rederive the answer. We'll start by making a guess.

$$f_0(x) = 1$$

This is a pretty bad guess. Specifically, we have $f'_0(x) = 0$ but we need $f'_0(x) = f_0(x) = 1$. 

Watch how we'll improve this iteratively:

$f_0(x) = 1 \\
f'_0(x) = 0$

$f_1(x) = 1+x \\
f_1'(x) = 1$

$f_2(x) = 1 + x + \frac{1}{2}x^2 \\
f'_2(x) = 1 + x$

$\vdots$

$f(x) = 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + \dots = \sum_{k=0}^\infin \frac{x^k}{k!}\\
f'(x) = 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + \dots = \sum_{k=0}^\infin \frac{x^k}{k!}$

This is indeed the Taylor expansion of $e^x$, but in a sense, this infinite sum is more fundamental than the idea of multiplying a number x-times. From the power rule of derivatives, we can see directly that $f(x) = f'(x)$. Notationally, we can keep the form of $e^x$, but semantically, we should really be referring to this sum. 

So $e^i$ is just $f(i)$. In other words:
$$e^i = 1 + i + \frac{-1}{2} + i\frac{-1}{3!} + \dots$$

The above is just equal to some complex number. It doesn't really simplify well. To extract some structure out of this, we can try evaluating $e^{i\theta}$ which is take the exponent of a complex number with magnitude $\theta$.

$$e^{i\theta} = 1 + i\theta + \frac{-\theta^2}{2} + i\frac{-\theta^3}{3!} + \dots \\
= \left(1 - \frac{\theta^2}{2} + \dots \right) + i\left(\theta - \frac{\theta^3}{3!} + \dots \right)$$

If you recall your Taylor Series, you may recognize the right hand side as the series for $\cos{\theta}$ and $\sin{\theta}$. Resulting in

$$e^{i\theta} = \cos{\theta} + i\sin{\theta}$$

### II) Euler's Equation from Vector Calculus
Here's an alternative method. Let's assume that $e^{i\theta}$ is some complex number. We can map every complex number to a vector in $\mathbb R^2$ by making the real part map to the x coordinate and the imaginary part map to the y coordinate.

Let $\theta$ be a function of time. In particular, let $\theta(t) = t$. This means we are just increasing $\theta$ over time at a constant rate. We'd like to know what sort of path that $e^{i\theta(t)}$ traces out over time.

What is the time derivative of $e^{i\theta}$? 
$$\frac{\text d}{\text{d}t}e^{i\theta} = i\dot\theta e^{i\theta} = i e^{i\theta}$$

So for some position vector, we have a velocity vector that is perpendicular to the position at all points in time. This is exactly the trajectory of a circle! How do we describe this trajectory directly? We'd say that the x component is some $\cos(\theta)$ and the y component is some $\sin(\theta)$. With the mapping we described before, we get:

$$e^{i\theta} = \cos{\theta} + i\sin{\theta}$$

## 2) Modeling a Single Sinusoid

A forward travelling, lossless voltage wave has the form
$$v(z,t) = |V_0^+|\cos(\omega t - \beta z + \phi_+)$$

Let's build up a geometric phasor understanding of this expression one piece at a time. Consider the following expression.
$$v(z,t) = \cos(\omega t)$$

Notice, from Euler's equation, this is equivalent to $\Re(e^{i\omega t})$, the real part of a complex exponential. What does this visually mean? 

Let's plot the complex number as a function of time. 

In [ ]:
magnitude_f = 1
phase_f = 0
omega = 1
beta = 0
pure_time = SinusoidSignal(magnitude_f, phase_f, beta, omega)

point = pure_time.space_time_vec[0,0]

fig, [axp, axt] = plt.subplots(1,2)
fig.subplots_adjust(bottom=0.3)

vec, = axp.plot([0, np.real(point)], [0, np.imag(point)])
real, = axp.plot([0, np.real(point)], [np.imag(point), np.imag(point)], "r")
dot, = axp.plot(np.real(point), np.imag(point), "ko")

axp.set_xlim([-2, 2])
axp.set_ylim([-2, 2])
axp.set_aspect("equal")
axp.set_title("Phasor Domain")
axp.set_xlabel("Real Part")
axp.set_ylabel("Imag Part")

vzt, = axt.plot(pure_time.space, pure_time.vzt[0])
sample_mag, = axt.plot([0, 0], [0, np.real(point)], "r")
samples, = axt.plot([0], [np.real(point)], "ok", ls="")

axt.set_ylim([-2, 2])
axt.set_title("Signal Across Space")
axt.set_xlabel("Space [m]")

ax_time = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=ax_time,
    label="Time (s)",
    valmin=0,
    valmax=10,
    valinit=0
)

def time_update(val):
    pure_time.current_time = val
    slider_update(pure_time.current_phasor[0])


def slider_update(point):
    vec.set_data([0, np.real(point)], [0, np.imag(point)])
    real.set_data([0, np.real(point)], [np.imag(point), np.imag(point)])
    dot.set_data([np.real(point)], [np.imag(point)])
    vzt.set_ydata(np.real(pure_time.current_phasor))
    sample_mag.set_ydata([0, np.real(point)])
    samples.set_ydata([np.real(point)])
    fig.canvas.draw_idle()

# register the update function with each slider
time_slider.on_changed(time_update)

Notice, as time progresses, the phasor (left plot, blue line with point at the end) moves in a circle (as expected from section 1). The real part of the phasor's coordinate (red) corresponds with the value of the time domain function (red). Because our function of voltage is only a function of time, it is constant through space.

Let's now look at what happens when we add in magnitudes and phase.
$$v(z,t) = |V_0^+|\cos(\omega t + \phi_+) = \Re (|V_0^+|e^{\omega t + \phi_+}) = \Re(V_0^+ e^{\omega t})$$

In [ ]:
magnitude_f = 1
phase_f = 0
omega = 1
beta = 0
pure_time = SinusoidSignal(magnitude_f, phase_f, beta, omega)

point = pure_time.space_time_vec[0,0]

fig, [axp, axt] = plt.subplots(1,2)
fig.subplots_adjust(bottom=0.3)

vec, = axp.plot([0, np.real(point)], [0, np.imag(point)])
real, = axp.plot([0, np.real(point)], [np.imag(point), np.imag(point)], "r")
dot, = axp.plot(np.real(point), np.imag(point), "ko")

axp.set_xlim([-2, 2])
axp.set_ylim([-2, 2])
axp.set_aspect("equal")
axp.set_title("Phasor Domain")
axp.set_xlabel("Real Part")
axp.set_ylabel("Imag Part")

vzt, = axt.plot(pure_time.space, pure_time.vzt[0])
sample_mag, = axt.plot([0, 0], [0, np.real(point)], "r")
samples, = axt.plot([0], [np.real(point)], "ok", ls="")

axt.set_ylim([-2, 2])
axt.set_title("Signal Across Space")
axt.set_xlabel("Space [m]")

max_phase = 4*np.pi
ax_phase = fig.add_axes([0.25, 0.06, 0.65, 0.03])
phase_slider = Slider(
    ax=ax_phase,
    label="Phase (rad)",
    valmin=0,
    valmax=max_phase,
    valinit=0
)

ax_mag = fig.add_axes([0.25, 0.03, 0.65, 0.03])
mag_slider = Slider(
    ax=ax_mag,
    label="Magnitude (V)",
    valmin=-2,
    valmax=2,
    valinit=1
)

ax_time = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=ax_time,
    label="Time (s)",
    valmin=0,
    valmax=10,
    valinit=0
)

def phase_update(val):
    pure_time.phase = val
    slider_update(pure_time.current_phasor[0])
def mag_update(val):
    pure_time.magnitude = val
    slider_update(pure_time.current_phasor[0])
def time_update(val):
    pure_time.current_time = val
    slider_update(pure_time.current_phasor[0])


def slider_update(point):
    vec.set_data([0, np.real(point)], [0, np.imag(point)])
    real.set_data([0, np.real(point)], [np.imag(point), np.imag(point)])
    dot.set_data([np.real(point)], [np.imag(point)])
    vzt.set_ydata(np.real(pure_time.current_phasor))
    sample_mag.set_ydata([0, np.real(point)])
    samples.set_ydata([np.real(point)])
    fig.canvas.draw_idle()

# register the update function with each slider
phase_slider.on_changed(phase_update)
mag_slider.on_changed(mag_update)
time_slider.on_changed(time_update)

Here, when $t=0$ we see the complex number $V_0^+$ plotted as a vector. Progressing in time rotates this vector. 

Changing the phase changes the initial angle of the phasor. This results in a different value in the time domain (red bar is changing). The magnitude term changes how long the phasor vector is. This changes the current magnitude in the time domain as well as the peak-to-peak magnitude.

Up until now, the phasor perspective has not really given us anything useful that we couldn't get from space-time domain signal. Before adding a spatial term, let's look at what happens when we add two cosine functions together. This will demonstrate the usefulness of the phasor representation. What does the following look like?
$$v(z, t) = M_1 \cos(\omega t + \phi_1) + M_2 \cos(\omega t + \phi_2) \\
= \Re(M_1 e^{\omega t + \phi_1}) + \Re(M_2 e^{\omega t + \phi_2}) \\
= \Re(M_1 e^{\omega t} e^{\phi_1} + M_2 e^{\omega t} e^{\phi_2}) \\
= \Re((M_1 e^{\phi_1} + M_2 e^{\phi_2})e^{\omega t})$$

This is similar to the previous situation. We have the sum of two complex numbers (which is just another complex number) times a $e^{\omega t}$ term. We should expect this behaves very similarly to the previous situation.

In [ ]:
magnitude_1 = 1
phase_1 = 0
magnitude_2 = 0.75
phase_2 = np.pi/3

omega = 1
beta = 0
pure_time_1 = SinusoidSignal(magnitude_1, phase_1, beta, omega)
pure_time_2 = SinusoidSignal(magnitude_2, phase_2, beta, omega)

point_1 = pure_time_1.space_time_vec[0,0]
point_2 = pure_time_2.space_time_vec[0,0]

fig, [axp, axt] = plt.subplots(1,2)
fig.subplots_adjust(bottom=0.3)

vec1, = axp.plot([0, np.real(point_1)], [0, np.imag(point_1)])
vec2, = axp.plot([0, np.real(point_2)], [0, np.imag(point_2)])
sup_1, = axp.plot([np.real(point_1), np.real(point_1+point_2)], [np.imag(point_1), np.imag(point_1+point_2)], "--k")
sup_2, = axp.plot([np.real(point_2), np.real(point_1+point_2)], [np.imag(point_2), np.imag(point_1+point_2)], "--k")
sum_vec, = axp.plot([0, np.real(point_1 + point_2)], [0, np.imag(point_1 + point_2)], "k")

real, = axp.plot([0, np.real(point_1+point_2)], [np.imag(point_1+point_2), np.imag(point_1+point_2)], "r")
dot, = axp.plot(np.real(point_1+point_2), np.imag(point_1+point_2), "ko")

axp.set_xlim([-2, 2])
axp.set_ylim([-2, 2])
axp.set_aspect("equal")
axp.set_title("Phasor Domain")
axp.set_xlabel("Real Part")
axp.set_ylabel("Imag Part")

vzt, = axt.plot(pure_time_1.space, pure_time_1.vzt[0] + pure_time_2.vzt[0])
sample_mag, = axt.plot([0, 0], [0, np.real(point_1 + point_2)], "r")
samples, = axt.plot([0], [np.real(point_1 + point_2)], "ok", ls="")

axt.set_ylim([-2, 2])
axt.set_title("Signal Across Space")
axt.set_xlabel("Space [m]")

ax_time = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=ax_time,
    label="Time (s)",
    valmin=0,
    valmax=10,
    valinit=0
)

def time_update(val):
    pure_time_1.current_time = val
    pure_time_2.current_time = val
    slider_update(pure_time_1.current_phasor[0], pure_time_2.current_phasor[0])


def slider_update(point_1, point_2):
    vec1.set_data([0, np.real(point_1)], [0, np.imag(point_1)])
    vec2.set_data([0, np.real(point_2)], [0, np.imag(point_2)])
    sup_1.set_data([np.real(point_1), np.real(point_1+point_2)], [np.imag(point_1), np.imag(point_1+point_2)])
    sup_2.set_data([np.real(point_2), np.real(point_1+point_2)], [np.imag(point_2), np.imag(point_1+point_2)])
    sum_vec.set_data([0, np.real(point_1+point_2)], [0, np.imag(point_1+point_2)])
    real.set_data([0, np.real(point_1 + point_2)], [np.imag(point_1 + point_2), np.imag(point_1 + point_2)])
    dot.set_data([np.real(point_1 + point_2)], [np.imag(point_1 + point_2)])
    vzt.set_ydata(np.real(pure_time_1.current_phasor + pure_time_2.current_phasor))
    sample_mag.set_ydata([0, np.real(point_1 + point_2)])
    samples.set_ydata([np.real(point_1 + point_2)])
    fig.canvas.draw_idle()

# register the update function with each slider
time_slider.on_changed(time_update)

We have two phasors that are being summed together. Summing the vectors results in a new vector (black solid line). The real part of this resultant vector is still equal to the magnitude of the time-domain signal. As this vector rotates, its real part is equivalent to the instantaneous magnitude of the time domain signal. As a result, what would typically be a relatively complicated, trig heavy operation is turned into ordinary vector addition and the calculation of a magnitude.

This is the key reason why phasor analysis is convenient and useful.

Let's now add in the spatial term and look at the fully general forward travelling wave equation.
$$v(z,t) = |V_0^+|\cos(\omega t - \beta z + \phi_+)$$

In [ ]:
magnitude_f = 1
phase_f = 0
omega = 1
beta = 3*np.pi
sinusoid = SinusoidSignal(magnitude_f, phase_f, beta, omega)

point = sinusoid.space_time_vec[0,0]

fig, [axp, axt] = plt.subplots(1,2)
fig.subplots_adjust(bottom=0.3)

vec, = axp.plot([0, np.real(point)], [0, np.imag(point)])
real, = axp.plot([0, np.real(point)], [np.imag(point), np.imag(point)], "r")
dot, = axp.plot(np.real(point), np.imag(point), "ko")

axp.set_xlim([-2, 2])
axp.set_ylim([-2, 2])
axp.set_aspect("equal")
axp.set_title("Phasor Domain")
axp.set_xlabel("Real Part")
axp.set_ylabel("Imag Part")

vzt, = axt.plot(sinusoid.space, sinusoid.vzt[0])
sample_mag, = axt.plot([0, 0], [0, np.real(point)], "r")
samples, = axt.plot([0], [np.real(point)], "ok", ls="")

axt.set_ylim([-2, 2])
axt.set_title("Signal Across Space")
axt.set_xlabel("Space [m]")

max_phase = 4*np.pi
ax_phase = fig.add_axes([0.25, 0.09, 0.65, 0.03])
phase_slider = Slider(
    ax=ax_phase,
    label="Phase (rad)",
    valmin=0,
    valmax=max_phase,
    valinit=0
)

ax_mag = fig.add_axes([0.25, 0.06, 0.65, 0.03])
mag_slider = Slider(
    ax=ax_mag,
    label="Magnitude (V)",
    valmin=-2,
    valmax=2,
    valinit=1
)

ax_space = fig.add_axes([0.25, 0.03, 0.65, 0.03])
space_slider = Slider(
    ax=ax_space,
    label="Space (m)",
    valmin=0,
    valmax=1,
    valinit=0
)

ax_time = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=ax_time,
    label="Time (s)",
    valmin=0,
    valmax=10,
    valinit=0
)

def phase_update(val):
    sinusoid.phase = val
    slider_update(sinusoid.current_phasor[0])
def mag_update(val):
    sinusoid.magnitude = val
    slider_update(sinusoid.current_phasor[0])
def space_update(val):
    sinusoid.current_loc = val
    slider_update(sinusoid.current_phasor[0])
def time_update(val):
    sinusoid.current_time = val
    slider_update(sinusoid.current_phasor[0])


def slider_update(point):
    phase_dom_point = sinusoid.space_time_vec[sinusoid._current_time_idx, sinusoid.current_loc_idx]
    vec.set_data([0, np.real(phase_dom_point)], [0, np.imag(phase_dom_point)])
    real.set_data([0, np.real(phase_dom_point)], [np.imag(phase_dom_point), np.imag(phase_dom_point)])
    dot.set_data([np.real(phase_dom_point)], [np.imag(phase_dom_point)])

    time_dom_point = sinusoid.vzt[sinusoid._current_time_idx, sinusoid.current_loc_idx]
    vzt.set_ydata(sinusoid.vzt[sinusoid._current_time_idx])
    sample_mag.set_data([sinusoid.current_loc, sinusoid.current_loc], [0, time_dom_point])
    samples.set_data([sinusoid.current_loc], [time_dom_point])
    fig.canvas.draw_idle()

# register the update function with each slider
phase_slider.on_changed(phase_update)
mag_slider.on_changed(mag_update)
space_slider.on_changed(space_update)
time_slider.on_changed(time_update)

Notice, progressing in space causes negative angular rotations in the phasor domain. This is because of the minus sign in front of the beta. 
$$v(z,t) = \Re(|V_0^+| e^{j (\omega t - \beta z + \phi_+)})$$

Intuitively, the minus sign is necessary for a forward travelling wave because moving forward in time and space at a fixed rate should ensure that $\omega t - \beta z = c$, that is if both increase at a rate that ensures their difference always remains constant, then the magnitude of the sinusoid should not change. This rate defines the phase velocity. 

Instead of looking at a single phase vector at a given point in time and space, it will be useful for us to instead look at all phase vectors across space at a particular point in time (a spatial contour if you will). Here's what that looks like.

In [ ]:
magnitude_f = 1
phase_f = 0
omega = 1
beta = 3*np.pi
sinusoid = SinusoidSignal(magnitude_f, phase_f, beta, omega)

point = sinusoid.space_time_vec[0,0]

fig, [axp, axt] = plt.subplots(1,2)
fig.subplots_adjust(bottom=0.3)

phase_contour, = axp.plot(np.real(sinusoid.phasor), np.imag(sinusoid.phasor), "k")
vec, = axp.plot([0, np.real(point)], [0, np.imag(point)])

real, = axp.plot([0, np.real(point)], [np.imag(point), np.imag(point)], "r")
dot, = axp.plot(np.real(point), np.imag(point), "ko")

axp.set_xlim([-2, 2])
axp.set_ylim([-2, 2])
axp.set_aspect("equal")
axp.set_title("Phasor Domain")
axp.set_xlabel("Real Part")
axp.set_ylabel("Imag Part")

vzt, = axt.plot(sinusoid.space, sinusoid.vzt[0])
sample_mag, = axt.plot([0, 0], [0, np.real(point)], "r")
samples, = axt.plot([0], [np.real(point)], "ok", ls="")

axt.set_ylim([-2, 2])
axt.set_title("Signal Across Space")
axt.set_xlabel("Space [m]")

max_phase = 4*np.pi
ax_phase = fig.add_axes([0.25, 0.09, 0.65, 0.03])
phase_slider = Slider(
    ax=ax_phase,
    label="Phase (rad)",
    valmin=0,
    valmax=max_phase,
    valinit=0
)

ax_mag = fig.add_axes([0.25, 0.06, 0.65, 0.03])
mag_slider = Slider(
    ax=ax_mag,
    label="Magnitude (V)",
    valmin=-2,
    valmax=2,
    valinit=1
)

ax_space = fig.add_axes([0.25, 0.03, 0.65, 0.03])
space_slider = Slider(
    ax=ax_space,
    label="Space (m)",
    valmin=0,
    valmax=1,
    valinit=0
)

ax_time = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=ax_time,
    label="Time (s)",
    valmin=0,
    valmax=10,
    valinit=0
)

def phase_update(val):
    sinusoid.phase = val
    slider_update(sinusoid.current_phasor[0])
def mag_update(val):
    sinusoid.magnitude = val
    slider_update(sinusoid.current_phasor[0])
def space_update(val):
    sinusoid.current_loc = val
    slider_update(sinusoid.current_phasor[0])
def time_update(val):
    sinusoid.current_time = val
    slider_update(sinusoid.current_phasor[0])


def slider_update(point):
    phase_dom_point = sinusoid.space_time_vec[sinusoid._current_time_idx, sinusoid.current_loc_idx]
    vec.set_data([0, np.real(phase_dom_point)], [0, np.imag(phase_dom_point)])
    phase_contour.set_data(np.real(sinusoid.phasor), np.imag(sinusoid.phasor))
    real.set_data([0, np.real(phase_dom_point)], [np.imag(phase_dom_point), np.imag(phase_dom_point)])
    dot.set_data([np.real(phase_dom_point)], [np.imag(phase_dom_point)])

    time_dom_point = sinusoid.vzt[sinusoid._current_time_idx, sinusoid.current_loc_idx]
    vzt.set_ydata(sinusoid.vzt[sinusoid._current_time_idx])
    sample_mag.set_data([sinusoid.current_loc, sinusoid.current_loc], [0, time_dom_point])
    samples.set_data([sinusoid.current_loc], [time_dom_point])
    fig.canvas.draw_idle()

# register the update function with each slider
phase_slider.on_changed(phase_update)
mag_slider.on_changed(mag_update)
space_slider.on_changed(space_update)
time_slider.on_changed(time_update)

For a single wave, this ends up being a circle. This makes sense, because our function is still of the form $Me^{j\theta}$ which traces out a circle. This will not be true when we consider the sum of forward and backwards travelling waves. 

Recall, the voltage of a forward and backwards travelling wave is as follows.
$$v(z,t) = |V_0^+|\cos(\omega t - \beta z + \phi_+) + |\Gamma||V_0^+|\cos(\omega t + \beta z + \phi_+ + \theta_r)$$

In [ ]:
omega = 1
beta = 3*np.pi

magnitude_f = 1
phase_f = 0
sinusoid_f = SinusoidSignal(magnitude_f, phase_f, beta, omega)

magnitude_r = 0.75
phase_r = 1
sinusoid_b = SinusoidSignal(magnitude_f * magnitude_r, phase_f + phase_r, -beta, omega)

point_f = sinusoid_f.space_time_vec[0,0]
point_b = sinusoid_b.space_time_vec[0,0]

fig, [axp, axt] = plt.subplots(1,2)
fig.subplots_adjust(bottom=0.3)

phase_contour, = axp.plot(np.real(sinusoid_f.phasor + sinusoid_b.phasor), np.imag(sinusoid_f.phasor + sinusoid_b.phasor), "k")
vec_f, = axp.plot([0, np.real(point_f)], [0, np.imag(point_f)])
vec_b, = axp.plot([0, np.real(point_b)], [0, np.imag(point_b)])
vec_sum, = axp.plot([0, np.real(point_b+point_f)], [0, np.imag(point_b+point_f)])


real, = axp.plot([0, np.real(point_f + point_b)], [np.imag(point_f + point_b), np.imag(point_f + point_b)], "r")
dot, = axp.plot(np.real(point_f + point_b), np.imag(point_f + point_b), "ko")

axp.set_xlim([-2, 2])
axp.set_ylim([-2, 2])
axp.set_aspect("equal")
axp.set_title("Phasor Domain")
axp.set_xlabel("Real Part")
axp.set_ylabel("Imag Part")

vzt, = axt.plot(sinusoid_f.space, sinusoid_f.vzt[0] + sinusoid_b.vzt[0])
sample_mag, = axt.plot([0, 0], [0, np.real(point_f + point_b)], "r")
samples, = axt.plot([0], [np.real(point_f + point_b)], "ok", ls="")

axt.set_ylim([-2, 2])
axt.set_title("Signal Across Space")
axt.set_xlabel("Space [m]")

ax_refl_phase = fig.add_axes([0.25, 0.15, 0.65, 0.03])
refl_phase_slider = Slider(
    ax=ax_refl_phase,
    label="Reflection Phase (rad)",
    valmin=0,
    valmax=np.pi * 2,
    valinit=phase_r
)

ax_refl_mag = fig.add_axes([0.25, 0.12, 0.65, 0.03])
refl_mag_slider = Slider(
    ax=ax_refl_mag,
    label="Reflection Magnitude",
    valmin=-1,
    valmax=1,
    valinit=magnitude_r
)

max_phase = 4*np.pi
ax_phase = fig.add_axes([0.25, 0.09, 0.65, 0.03])
phase_slider = Slider(
    ax=ax_phase,
    label="Phase (rad)",
    valmin=0,
    valmax=max_phase,
    valinit=0
)

ax_mag = fig.add_axes([0.25, 0.06, 0.65, 0.03])
mag_slider = Slider(
    ax=ax_mag,
    label="Magnitude (V)",
    valmin=-2,
    valmax=2,
    valinit=1
)

ax_space = fig.add_axes([0.25, 0.03, 0.65, 0.03])
space_slider = Slider(
    ax=ax_space,
    label="Space (m)",
    valmin=0,
    valmax=1,
    valinit=0
)

ax_time = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=ax_time,
    label="Time (s)",
    valmin=0,
    valmax=10,
    valinit=0
)

def refl_phase_update(val):
    sinusoid_b.phase = sinusoid_f.phase + val
    slider_update()
def refl_mag_update(val):
    sinusoid_b.magnitude = sinusoid_f.magnitude * val
    slider_update()
def phase_update(val):
    phase_r = sinusoid_b.phase - sinusoid_f.phase
    sinusoid_f.phase = val
    sinusoid_b.phase = val + phase_r
    slider_update()
def mag_update(val):
    magnitude_r = sinusoid_b.magnitude / sinusoid_f.magnitude
    sinusoid_f.magnitude = val
    sinusoid_b.magnitude = val * magnitude_r
    slider_update()
def space_update(val):
    sinusoid_f.current_loc = val
    sinusoid_b.current_loc = val
    slider_update()
def time_update(val):
    sinusoid_f.current_time = val
    sinusoid_b.current_time = val
    slider_update()


def slider_update():
    phase_dom_point_f = sinusoid_f.space_time_vec[sinusoid_f._current_time_idx, sinusoid_f.current_loc_idx]
    phase_dom_point_b = sinusoid_b.space_time_vec[sinusoid_b._current_time_idx, sinusoid_b.current_loc_idx]
    phase_dom_point = phase_dom_point_f + phase_dom_point_b
    space_time_sum = sinusoid_f.space_time_vec[sinusoid_f._current_time_idx, :] + sinusoid_b.space_time_vec[sinusoid_b._current_time_idx, :]
    
    phase_contour.set_data(np.real(space_time_sum), np.imag(space_time_sum))

    vec_f.set_data([0, np.real(phase_dom_point_f)], [0, np.imag(phase_dom_point_f)])
    vec_b.set_data([0, np.real(phase_dom_point_b)], [0, np.imag(phase_dom_point_b)])
    vec_sum.set_data([0, np.real(phase_dom_point)], [0, np.imag(phase_dom_point)])

    real.set_data([0, np.real(phase_dom_point)], [np.imag(phase_dom_point), np.imag(phase_dom_point)])
    dot.set_data([np.real(phase_dom_point)], [np.imag(phase_dom_point)])

    vzt.set_ydata(sinusoid_f.vzt[sinusoid_f._current_time_idx] + sinusoid_b.vzt[sinusoid_b._current_time_idx])
    # sample_mag, = axt.plot([0, 0], [0, np.real(point_f + point_b)], "r")
    # samples, = axt.plot([0], [np.real(point_f + point_b)], "ok", ls="")

    time_dom_point = sinusoid_f.vzt[sinusoid_f._current_time_idx, sinusoid_f.current_loc_idx] + sinusoid_b.vzt[sinusoid_b._current_time_idx, sinusoid_b.current_loc_idx]
    # vzt.set_ydata(sinusoid.vzt[sinusoid._current_time_idx])
    sample_mag.set_data([sinusoid_f.current_loc, sinusoid_f.current_loc], [0, time_dom_point])
    samples.set_data([sinusoid_f.current_loc], [time_dom_point])
    fig.canvas.draw_idle()

# register the update function with each slider
refl_phase_slider.on_changed(refl_phase_update)
refl_mag_slider.on_changed(refl_mag_update)
phase_slider.on_changed(phase_update)
mag_slider.on_changed(mag_update)
space_slider.on_changed(space_update)
time_slider.on_changed(time_update)

The forward phasor vector is blue, the backwards phasor vector is dark red. Their sum is in purple. The sum traces the spatial contour. Notice how a discrepancy in magnitude of the forward and backward phasor results in an elliptical spatial contour. 

A few observations
- Changing the reflection phase does not change the shape of the spatial contour, just rotates it
- When the reflection magnitude is zero, the spatial contour reduces to a circle and the situation is the same as only having a forward travelling wave.
- The reflection magnitude changes the shape of the spatial contour. This is what affects the behavior of the time domain sinusoid.
- Time causes the phasor to always travel in a circle. This is due to the fact that both forward and backwards waves have the same $\omega$ with the same sign. 